## Pt1: Get the Weather Description and Amount of Precipitation for Each City



#### Generate a new set of 1,500 random latitudes and longitudes.

In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-70.000, high=70.000, size=1500)
lngs = np.random.uniform(low=-140.000, high=140.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

#### Get the nearest city using the citipy module.

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

741

#### Perform an API call with the OpenWeatherMap.

In [6]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_keys

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791


In [8]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791&q=San Francisco



#### Make a Request for Data to an API

In [9]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
city_weather = requests.get(city_url)
city_weather

<Response [200]>

In [10]:
# Make a 'Get' request for the city weather. Use the get() HTTP method 
city_weather.status_code

200

In [11]:
# Get the JSON format of the request; Get data from a response    
city_weather.json()

{'coord': {'lon': -76.88, 'lat': 1.17},
 'weather': [{'id': 521,
   'main': 'Rain',
   'description': 'shower rain',
   'icon': '09n'}],
 'base': 'stations',
 'main': {'temp': 66.2,
  'pressure': 1017,
  'humidity': 88,
  'temp_min': 66.2,
  'temp_max': 66.2},
 'visibility': 10000,
 'wind': {'speed': 4.7, 'deg': 20},
 'clouds': {'all': 75},
 'dt': 1573342234,
 'sys': {'type': 1,
  'id': 8592,
  'country': 'CO',
  'sunrise': 1573296559,
  'sunset': 1573340005},
 'timezone': -18000,
 'id': 3669881,
 'name': 'San Francisco',
 'cod': 200}

In [12]:
# Get information from the JSON formatted data.
San_Francisco_data = city_weather.json()

In [13]:
# Retrieve data: Latitude and longitude, Maximum temperature, Percent humidity, Percent cloudiness, Wind speed, 
# Weather description(e.g.clouds, fog, light rain, clear sky)

lat = San_Francisco_data["coord"]["lat"]
lng = San_Francisco_data["coord"]["lon"]
max_temp = San_Francisco_data["main"]["temp_max"]
humidity = San_Francisco_data["main"]["humidity"]
clouds = San_Francisco_data["clouds"]["all"]
wind = San_Francisco_data["wind"]["speed"]
weather_description = San_Francisco_data['weather'][0]['description']
#rain_inches = San_Francisco_data['rain']['3h']
#snow_inches = San_Francisco_data['snow']['3h']
print(lat, lng, max_temp, humidity, clouds, wind, weather_description) #rain_inches, snow_inches)

1.17 -76.88 66.2 88 75 4.7 shower rain



#### Handle Request Errors

###### Evaluate whether the status code is equal to 200 and is therefore a valid response

In [14]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "San Francisco"
city_weather = requests.get(city_url)
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")
    
print(city_url)

City Weather found.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8ddc114b26221886571a6d9338b01791&q=San Francisco



#### Get the amount of rainfall in inches for the last three hours

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1       
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        
        
    # Using a try-except block, if it is raining,snowing get the amount of rainfall or snow in inches for the last three hours. 
    # If it is not raining, add 0 inches for the city.
        
        try:
            city_rain_inches = city_weather['rain']['3h']
        except(KeyError):
            city_rain_inches = 0
        try:
            city_snow_inches = city_weather['snow']['3h']
        except(KeyError):
            city_snow_inches = 0
            
            
         # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description":city_weather_description,
                          "Rain Inches(last 3hrs)":city_rain_inches,
                          "Snow Inches(last 3hrs)":city_snow_inches,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kankakee
Processing Record 2 of Set 1 | plettenberg bay
Processing Record 3 of Set 1 | lishui
Processing Record 4 of Set 1 | saint-philippe
Processing Record 5 of Set 1 | baijiantan
Processing Record 6 of Set 1 | san carlos de bariloche
Processing Record 7 of Set 1 | khuzhir
Processing Record 8 of Set 1 | deer lake
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | westport
Processing Record 11 of Set 1 | esperance
Processing Record 12 of Set 1 | shymkent
Processing Record 13 of Set 1 | umm lajj
Processing Record 14 of Set 1 | port lincoln
Processing Record 15 of Set 1 | flinders
Processing Record 16 of Set 1 | cape town
Processing Record 17 of Set 1 | iskateley
Processing Record 18 of Set 1 | atuona
Processing Record 19 of Set 1 | rochester
Processing Record 20 of Set 1 | pochutla
Processing Record 21 of Set 1 | busselton
Processing Record 22 of Set 1 | victoria
Processing Re

Processing Record 35 of Set 4 | abrau-dyurso
Processing Record 36 of Set 4 | casablanca
Processing Record 37 of Set 4 | liuzhou
Processing Record 38 of Set 4 | saint-joseph
Processing Record 39 of Set 4 | san quintin
Processing Record 40 of Set 4 | lagoa
Processing Record 41 of Set 4 | bagan
Processing Record 42 of Set 4 | mar del plata
Processing Record 43 of Set 4 | louisbourg
City not found. Skipping...
Processing Record 44 of Set 4 | agutaya
Processing Record 45 of Set 4 | nisia floresta
Processing Record 46 of Set 4 | eureka
Processing Record 47 of Set 4 | caxito
Processing Record 48 of Set 4 | vila franca do campo
Processing Record 49 of Set 4 | blagoyevo
Processing Record 50 of Set 4 | tabou
Processing Record 1 of Set 5 | atambua
Processing Record 2 of Set 5 | babu
Processing Record 3 of Set 5 | hay river
Processing Record 4 of Set 5 | acari
Processing Record 5 of Set 5 | houma
Processing Record 6 of Set 5 | los llanos de aridane
Processing Record 7 of Set 5 | coquimbo
Processin

Processing Record 26 of Set 8 | lumphat
Processing Record 27 of Set 8 | hami
Processing Record 28 of Set 8 | padang
Processing Record 29 of Set 8 | teguldet
Processing Record 30 of Set 8 | lastoursville
Processing Record 31 of Set 8 | gigmoto
Processing Record 32 of Set 8 | luwuk
Processing Record 33 of Set 8 | khani
Processing Record 34 of Set 8 | tomatlan
Processing Record 35 of Set 8 | fergus falls
Processing Record 36 of Set 8 | burica
City not found. Skipping...
Processing Record 37 of Set 8 | zhigansk
Processing Record 38 of Set 8 | kulhudhuffushi
Processing Record 39 of Set 8 | linjiang
Processing Record 40 of Set 8 | nevel
Processing Record 41 of Set 8 | sao geraldo do araguaia
Processing Record 42 of Set 8 | kangaatsiaq
Processing Record 43 of Set 8 | denpasar
Processing Record 44 of Set 8 | vredendal
Processing Record 45 of Set 8 | asheville
Processing Record 46 of Set 8 | danville
Processing Record 47 of Set 8 | port elizabeth
Processing Record 48 of Set 8 | xining
Processin

#### Add the data to a new DataFrame.

In [ ]:
# Convert a list of dictionaries to a Pandas DataFrame using df = pd.DataFrame(list with dictionaries).
# Convert the array of dictionaries to a Pandas DataFrame.
weather_database_df = pd.DataFrame(city_data)
weather_database_df.head(10)

In [ ]:
# Reorder the columns as City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, and Wind Speed, Weather Description,
# Rain inches(last 3hrs), Snow inches(last 3 hrs)

new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 
                    'Weather Description','Rain Inches(last 3hrs)', 'Snow Inches(last 3hrs)']

# Assign a new or the same DataFrame with new column order: df = df[new_column_order]
weather_database_df = weather_database_df[new_column_order]
weather_database_df.head(15)

In [ ]:
# Save the new DF as a CSV file or Create the output file (CSV).

output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the City_Data into a CSV.
weather_database_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
#Answer this question using Pandas methods: How many cities have recorded snow?
snow_record= weather_database_df.loc[weather_database_df["Snow Inches(last 3hrs)"] > 0]
len(snow_record)

In [ ]:
rain_record= weather_database_df.loc[weather_database_df["Rain Inches(last 3hrs)"] > 0]
len(rain_record)

In [ ]:
snow_record

In [ ]:
rain_record